In [ ]:
import qsharp

In [ ]:
from collections import Counter
from itertools import product
import plotly.graph_objects as go
from qsharp import Result

def plot(
    results: list,
    title: str
) -> None:
    first = results[0] if results else None
    if isinstance(first, (list, tuple)):
        n_qubits = len(first)
        bitstrings = [
            "".join("0" if r == Result.Zero else "1" for r in shot)
            for shot in results
        ]
        counts = Counter(bitstrings)
        all_strs = ["".join(p) for p in product("01", repeat=n_qubits)]
        for bs in all_strs:
            counts.setdefault(bs, 0)
        outcomes = all_strs
    else:
        labels = ["Zero" if r == Result.Zero else "One" for r in results]
        counts = Counter(labels)
        for key in ("Zero", "One"):
            counts.setdefault(key, 0)
        outcomes = ["Zero", "One"]

    freqs = [counts[o] for o in outcomes]

    fig = go.Figure(
        data=go.Bar(
            x=outcomes,
            y=freqs,
            text=freqs,
        ),
        layout=go.Layout(
            title=title,
        )
    )
    fig.show()


In [ ]:
%%qsharp

operation PoloniumAtomAndObserver() : Result {
    // qubit in a superposition pretends to be a polonium atom
    use atom = Qubit();
    H(atom);

    // measurement simulates the observer
    let decay = M(atom);
    Reset(atom);
    return decay;
}

In [ ]:
results = qsharp.run("PoloniumAtomAndObserver()", shots=1000)
plot(results, title="Polonium Atom: Observed Decay Outcomes")

In [ ]:
%%qsharp

operation AtomCatAndObserver() : Result[] {
    // qubit in a superposition pretends to be a polonium atom
    use atom = Qubit();
    H(atom);

    // another qubit pretends to be a cat
    // it is entangled with the atom
    use cat = Qubit();
    CNOT(atom, cat);

    // measurement simulates the observer
    let results = MResetEachZ([atom, cat]);
    return results;
}

In [ ]:
results = qsharp.run("AtomCatAndObserver()", shots=1000)
plot(results, title="Atom and Cat Entanglement")

In [ ]:
%%qsharp

operation AtomCatObserverAndEnvironment() : Result[] {
    // qubit in a superposition pretends to be a polonium atom
    use atom = Qubit();
    H(atom);

    // another qubit pretends to be a cat
    // it is entangled with the atom
    use cat = Qubit();
    CNOT(atom, cat);

    // 3 more qubits pretend to be the environment, also entangled
    use env = Qubit[3];
    for i in 0..2 {
        CNOT(cat, env[i]);
    }

    // measurement simulates the observer
    let results = MResetEachZ([atom, cat] + env);
    return results;
}

In [ ]:
results = qsharp.run("AtomCatObserverAndEnvironment()", shots=1000)
plot(results, title="Atom, Cat, Observer and Environment Entanglement")